In [ ]:
# Allowed to make changes.
import collections
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
!pip install transformers
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering,AutoModel,AutoConfig
from transformers import AutoTokenizer,AutoModelForQuestionAnswering, TrainingArguments, Trainer,AutoConfig,AutoModel
from transformers.modeling_outputs import QuestionAnsweringModelOutput
from transformers import PreTrainedModel,PretrainedConfig
!pip install gensim==4.0.1
from gensim.parsing.preprocessing import preprocess_string
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

from gensim.models import Word2Vec, Doc2Vec
import gensim.downloader
import numpy as np
from sklearn.neighbors import NearestNeighbors

!pip install rank_bm25
from rank_bm25 import BM25Okapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 11.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 53.4 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


/usr/local/lib/python3.8/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Use gdrive temporarily, replace with wget

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
TASK1_CKPT = '/content/drive/MyDrive/InterIIT-DevRev/miniLM/task1_minilm'
TASK2_CKPT = '/content/drive/MyDrive/InterIIT-DevRev/miniLM/task2_minilm_ckpt-5076'#'/content/drive/MyDrive/InterIIT-DevRev/checkpoint-5076'
TASK2_CKPT_FT = '/content/drive/MyDrive/InterIIT-DevRev/miniLM/checkpoint-1200'
TEST_DATA_PATH = '/content/drive/MyDrive/InterIIT-DevRev/data/Task2dataSet_test.csv'
FULL_DATA = '/content/drive/MyDrive/InterIIT-DevRev/data/augment_data.csv'

In [ ]:
# Allowed to make changes.
PARAGRAPHS_PATH = '/content/drive/MyDrive/InterIIT-DevRev/data/10th_data/paragraphs.csv'
para_df = pd.read_csv(PARAGRAPHS_PATH)
paragraph_ds = para_df['paragraph'].tolist()
class BM_retriever():
  def __init__(self,paragraphs,para_ids=None):
    self.paras = paragraphs
    # self.para_ids = para_ids
    print("Pre-processing paragraphs")
    tokenized_corpus= [preprocess_string(t) for t in self.paras]
    print("Loading BM")
    self.bm25 = BM25Okapi(tokenized_corpus)

  def retrieve(self, question,k=10):
    tokenized_query = preprocess_string(question)
    doc_scores = self.bm25.get_scores(tokenized_query)
    pred_para_ids = np.argsort(doc_scores, axis=0)[-k:][::-1]
    return pred_para_ids, [self.paras[p] for p  in pred_para_ids[:k]]


class DistillBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistillBERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
        self.pre_classifier = torch.nn.Linear(384, 384)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(384, 2)

    def forward(self, input_ids, attention_mask):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

class MiniLMQA(PreTrainedModel):
    def __init__(self,config: PretrainedConfig):
        # super(DistillBERTQA, config).__init__()
        super().__init__(config)
        self.MiniLM = AutoModel.from_pretrained("microsoft/MiniLM-L12-H384-uncased") #DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.qa_outputs = torch.nn.Linear(384, 2)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, input_ids=None, attention_mask=None,start_positions=None,end_positions=None,return_dict=None):
        minilm_output = self.MiniLM(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = minilm_output[0]
        hidden_states = self.dropout(hidden_states)
        logits = self.qa_outputs(hidden_states)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()  # (bs, max_query_len)
        end_logits = end_logits.squeeze(-1).contiguous()  # (bs, max_query_len)
        total_loss = None
        if start_positions is not None and end_positions is not None:
            # If we are on multi-GPU, split add a dimension
            if len(start_positions.size()) > 1:
                start_positions = start_positions.squeeze(-1)
            if len(end_positions.size()) > 1:
                end_positions = end_positions.squeeze(-1)
            # sometimes the start/end positions are outside our model inputs, we ignore these terms
            ignored_index = start_logits.size(1)
            start_positions = start_positions.clamp(0, ignored_index)
            end_positions = end_positions.clamp(0, ignored_index)

            loss_fct = torch.nn.CrossEntropyLoss(ignore_index=ignored_index)
            start_loss = loss_fct(start_logits, start_positions)
            end_loss = loss_fct(end_logits, end_positions)
            total_loss = (start_loss + end_loss) / 2

        if not return_dict:
            output = (start_logits, end_logits) + minilm_output[1:]
            return ((total_loss,) + output) if total_loss is not None else output

        return QuestionAnsweringModelOutput(
            loss=total_loss,
            start_logits=start_logits,
            end_logits=end_logits,
            hidden_states=minilm_output.hidden_states,
            attentions=minilm_output.attentions,
        )

############# Task1 model #############
print("Loading task1 model...")
task1_model = DistillBERTClass()
task1_tokenizer = AutoTokenizer.from_pretrained(TASK1_CKPT)
task1_statedict = torch.load(TASK1_CKPT + '/pytorch_distilbert.bin',map_location=torch.device('cpu'))
task1_model.load_state_dict(task1_statedict.state_dict())
print("Loaded task1 model")
########################################


############# Task2 model #############
print("Loading task2 model...")
task2_tokenizer = AutoTokenizer.from_pretrained(TASK2_CKPT_FT)
task2_config = AutoConfig.from_pretrained(TASK2_CKPT)
task2_model = MiniLMQA(task2_config)
task2_state_dict = torch.load(TASK2_CKPT + '/pytorch_model.bin',map_location=torch.device('cpu'))
task2_model.load_state_dict(task2_state_dict)
print("Loaded task2 model")
########################################

############# Combined model #############
# for now dictionary and seperate loading
global_model = {'task1':{'model':task1_model,'tokenizer':task1_tokenizer},'task2':{'model':task2_model,'tokenizer':task2_tokenizer}}
########################################

############ BM model #############
print("Loading BM model...")
BM_model = BM_retriever(paragraph_ds)
print("Loaded BM model")
############ BM model #############


def answerable(model,tokenizer,question,context):
  concat_str = question +' <sep> '+ context
  inputs = tokenizer.encode_plus(
            concat_str,
            None,
            add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True,
            return_tensors = 'pt'
        )
  model_inputs = inputs
  del model_inputs['token_type_ids']
  pred = torch.argmax(model(**model_inputs),dim=1)
  if(pred == 1):
    return True
  else:
    return False

def get_answer(model,tokenizer,question,context):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt",max_length=512,truncation=True)
    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    model_inputs = inputs
    del model_inputs['token_type_ids']
    output = model(**model_inputs)

    answer_start_scores, answer_end_scores,_ = model(**inputs)

    answer_start = torch.argmax(
        answer_start_scores
    )  # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # Get the most likely end of answer with the argmax of the score

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    return answer

def run_prediction(model,question,context):
  if(answerable(model = model['task1']['model'],tokenizer = model['task1']['tokenizer'],question = question,context = context)):
    return get_answer(model = model['task2']['model'],tokenizer = model['task2']['tokenizer'],question = question,context = context)
  else:
    return ""


Loading task1 model...


Loaded task1 model
Loading task2 model...
Loaded task2 model
Loading BM model...
Pre-processing paragraphs
Loading BM
Loaded BM model


In [ ]:
# Allowed to make changes.
def get_theme_model(theme):
  # Here, you can load and return a model which is fine tuned for a theme.
  # In case, you only have a global model, you can return that.
  return global_model

In [ ]:

def retrieve_contestants_BM(question,k=3):
  ret_para,ret_para_ids = BM_model.retrieve(question,k)
  return (ret_para,ret_para_ids)

def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]["theme"]
  for question in questions:
    ans = {}
    ans["question_id"] = question["id"]
    questions_str = question['question']
    contensting_paras_ids,contensting_paras = retrieve_contestants_BM(question=questions_str,k=10)
    for i in range(len(contensting_paras)):
      para = contensting_paras[i]
      para_id = contensting_paras_ids[i]
      our_prediction = run_prediction(model = theme_model,question=questions_str,context=para)
      ans["answers"] = our_prediction
      if(our_prediction == ""):
        ans["paragraph_id"] = -1
      else:
        ans["paragraph_id"] = para_id
        break
    pred_out.append(ans)

In [ ]:
# NOT allowed to make changes.

# All theme prediction.
questions = json.loads(pd.read_csv("/content/sample_input_question_1.csv").to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv("/content/sample_theme_interval.csv").to_json(orient="records"))
pred_out = []
theme_inf_time = {}
for theme_interval in theme_intervals:
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
pred_df.fillna(value='', inplace=True)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv('/content/sample_output_prediction.csv', index=False)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
# NOT allowed to make changes.

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def calc_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def calc_max_f1(predicted, ground_truths):
  max_f1 = 0
  if len(ground_truths) == 0:
    return len(predicted) == 0
  for ground_truth in ground_truths:
    f1 = calc_f1(predicted, ground_truth)
    max_f1 = max(max_f1, f1)
  return max_f1

In [ ]:
# NOT allowed to make changes.

# Evaluation methodology.
metrics = {}
pred = pd.read_csv("sample_output_prediction.csv")
pred.fillna(value='', inplace=True)
truth = pd.read_csv("sample_ground_truth.csv")
truth.fillna(value='', inplace=True)
truth.paragraph_id = truth.paragraph_id.apply(literal_eval)
truth.answers = truth.answers.apply(literal_eval)
questions = pd.read_csv("sample_input_question.csv")
for idx in pred.index:
  q_id = pred["question_id"][idx]
  q_rows = questions.loc[questions['id'] == q_id].iloc[-1]
  theme = q_rows["theme"]
  predicted_paragraph = pred["paragraph_id"][idx]
  predicted_ans = pred["answers"][idx]

  if theme not in metrics.keys():
    metrics[theme] = {"true_positive": 0, "true_negative": 0, "total_predictions": 0, "f1_sum": 0}

  truth_row = truth.loc[truth['question_id'] == q_id].iloc[-1]
  truth_paragraph_id = [ int(i) for i in truth_row["paragraph_id"] ]
  if predicted_paragraph in truth_paragraph_id:
    # Increase TP for that theme.
    metrics[theme]["true_positive"] = metrics[theme]["true_positive"] + 1
  # -1 prediction in case there is no paragraph which can answer the query.
  if predicted_paragraph == -1 and truth_row["paragraph_id"] == []:
    # Increase TN.
    metrics[theme]["true_negative"] = metrics[theme]["true_negative"] + 1
  # Increase total predictions for that theme.
  metrics[theme]["total_predictions"] = metrics[theme]["total_predictions"] + 1
  f1 = calc_max_f1(predicted_ans, truth_row["answers"])
  metrics[theme]["f1_sum"] = metrics[theme]["f1_sum"] + f1

In [ ]:
# NOT allowed to make changes.

# Final score.
inf_time_threshold = 1000.0 # milliseconds.
final_para_score = 0.0
final_qa_score = 0.0
# Weight would stay hidden from teams.
theme_weights = {"Kubernetes": 0.5, "ChatGPT": 0.4, "Football world cup": 0.1}
for theme in metrics:
  inf_time_score = 1.0
  metric = metrics[theme]
  para_score = (metric["true_positive"] + metric["true_negative"]) / metric["total_predictions"]
  qa_score = metric["f1_sum"] / metric["total_predictions"]
  avg_inf_time = theme_inf_time[theme] / metric["total_predictions"]
  if avg_inf_time > inf_time_threshold:
    inf_time_score = inf_time_threshold / avg_inf_time
  final_qa_score += theme_weights[theme] * inf_time_score * qa_score
  final_para_score += theme_weights[theme] * inf_time_score * para_score
print (final_para_score)
print (final_qa_score)


0.7
0.5666666666666667
